In [126]:
from qubolite import qubo
import numpy as np

In [127]:
import sys
sys.path.insert(1, '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/Deep-Opt')

In [128]:
import torch

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

In [129]:
# Highly recommended to keep as cpu for problems of size <= 100
device="cpu"
print(device)
device = torch.device(device)

cpu


In [130]:
problem = QUBO("/Users/linusrandud/Documents/UoM/ERP/MscDissertation/data/qubo/qubo_problem.txt", 0, device)

Instance has been loaded


In [131]:
# original

# a = [13, 23, 16, 20]
# solution should be [0, 0, 1, 1] or [1, 1, 0, 0]

In [132]:
problem.Q.detach().cpu().numpy()

array([[-3731.,   299.,   169.,   169.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  299., -6371.,   299.,   299.,   299.,   299.,   299.,   299.,
          529.,   299.,   230.,   230.,   230.,   460.,   575.,   460.,
          230.,   345.,   230.,   460.],
       [  169.,   299., -3731.,   169.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  169.,   299.,   169., -3731.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  169.,   299.,   169.,   169., -3731.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [  169.,   299.,   169.,   169.,   169., -3731.,   169.,   169.,
   

In [133]:
np.triu(problem.Q.detach().cpu().numpy())

array([[-3731.,   299.,   169.,   169.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [    0., -6371.,   299.,   299.,   299.,   299.,   299.,   299.,
          529.,   299.,   230.,   230.,   230.,   460.,   575.,   460.,
          230.,   345.,   230.,   460.],
       [    0.,     0., -3731.,   169.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [    0.,     0.,     0., -3731.,   169.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [    0.,     0.,     0.,     0., -3731.,   169.,   169.,   169.,
          299.,   169.,   130.,   130.,   130.,   260.,   325.,   260.,
          130.,   195.,   130.,   260.],
       [    0.,     0.,     0.,     0.,     0., -3731.,   169.,   169.,
   

In [134]:
Q = problem.Q.detach().cpu().numpy()

In [135]:
from qubolite.solving import brute_force, local2_descent, local2_descent_search, simulated_annealing

In [136]:
Q_ = qubo(Q)
# brute_force(Q_)

In [137]:
local2_descent(Q_)

qubo_solution(x=array([ True,  True, False, False,  True, False, False,  True,  True,
       False,  True, False, False,  True,  True, False,  True, False,
       False, False]), energy=-22500.0)

In [138]:
local2_descent_search(Q_)

qubo_solution(x=array([0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1.,
       0., 0., 0.]), energy=-22500.0)

In [139]:
simulated_annealing(Q_)

Init. temp. automatically set to 82039.1882


(array([[1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
         1., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
         1., 0., 1., 0.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0.,
         0., 0., 1., 0.],
        [0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1.],
        [1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1.,
         1., 0., 0., 0.],
        [0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1.,
         1., 1., 1., 0.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         1., 1., 1., 1.],
        [0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         1., 0., 0., 0.],
        [1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0.]]),
 array([

In [140]:
def evaluate_qubo(Q):
    n = Q.shape[0]
    min_value = float('inf')
    best_x = None
    
    for i in range(2**n):
        x = np.array(list(map(int, bin(i)[2:].zfill(n))))
        value = x.T @ Q @ x
        if value < min_value:
            min_value = value
            best_x = x
    
    return best_x, min_value

In [141]:
evaluate_qubo(Q)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), -22500.0)